# Neo4J Openai Langchain quering Graph Database

### GraphCypherQAChain
https://api.python.langchain.com/en/latest/chains/langchain.chains.graph_qa.cypher.GraphCypherQAChain.html
### Neo4jGraph
https://python.langchain.com/docs/integrations/graphs/neo4j_cypher/

### Integrations Langchain with Graph Databases

https://python.langchain.com/docs/integrations/graphs/

In [1]:
from dotenv import dotenv_values
config = dotenv_values(".env")
KEY_OPENAI = config.get("OPENAI_API_KEY")

In [22]:
from langchain_openai import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    openai_api_key=KEY_OPENAI,model ="gpt-4"
)

graph = Neo4jGraph(
    url="bolt://18.232.137.29:7687",
    username="neo4j",
    password="counters-transport-tension",
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

cypher_chain.run("In which movies  acted Tom Hanks? return the number and a list of them")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie)
RETURN COUNT(m) as NumberOfMovies, COLLECT(m.title) as MovieList
Full Context:
[{'NumberOfMovies': 12, 'MovieList': ['Apollo 13', "You've Got Mail", 'A League of Their Own', 'Joe Versus the Volcano', 'That Thing You Do', 'The Da Vinci Code', 'Cloud Atlas', 'Cast Away', 'The Green Mile', 'Sleepless in Seattle', 'The Polar Express', "Charlie Wilson's War"]}]

> Finished chain.


'Tom Hanks acted in 12 movies. The list includes \'Apollo 13\', "You\'ve Got Mail", \'A League of Their Own\', \'Joe Versus the Volcano\', \'That Thing You Do\', \'The Da Vinci Code\', \'Cloud Atlas\', \'Cast Away\', \'The Green Mile\', \'Sleepless in Seattle\', \'The Polar Express\', and "Charlie Wilson\'s War".'

In [23]:
cypher_chain.run("What movies did Meg Ryan act in? return the count and list of names")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Meg Ryan"})-[:ACTED_IN]->(m:Movie)
RETURN COUNT(m) AS count, COLLECT(m.title) AS movies
Full Context:
[{'count': 5, 'movies': ['Sleepless in Seattle', "You've Got Mail", 'Top Gun', 'When Harry Met Sally', 'Joe Versus the Volcano']}]

> Finished chain.


'Meg Ryan acted in 5 movies. These include \'Sleepless in Seattle\', "You\'ve Got Mail", \'Top Gun\', \'When Harry Met Sally\', and \'Joe Versus the Volcano\'.'

In [24]:
cypher_chain.run( "How many movies has Tom Hanks directed? tell me as well the titles")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:Person {name: 'Tom Hanks'})-[:DIRECTED]->(m:Movie)
RETURN COUNT(m) AS TotalMoviesDirected, COLLECT(m.title) AS MovieTitles

Full Context:
[{'TotalMoviesDirected': 1, 'MovieTitles': ['That Thing You Do']}]

> Finished chain.


"Tom Hanks has directed 1 movie, which is titled 'That Thing You Do'."

In [25]:
cypher_chain.run( "In which Movies acted Tom Hanks and Meg Ryan together?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person {name: 'Meg Ryan'})
RETURN m.title
Full Context:
[{'m.title': 'Sleepless in Seattle'}, {'m.title': "You've Got Mail"}, {'m.title': 'Joe Versus the Volcano'}]

> Finished chain.


'Tom Hanks and Meg Ryan acted together in \'Sleepless in Seattle\', "You\'ve Got Mail", and \'Joe Versus the Volcano\'.'

In [26]:
cypher_chain.run("Who played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[{'p.name': 'Val Kilmer'}, {'p.name': 'Meg Ryan'}, {'p.name': 'Tom Skerritt'}, {'p.name': 'Kelly McGillis'}, {'p.name': 'Tom Cruise'}, {'p.name': 'Anthony Edwards'}]

> Finished chain.


'Val Kilmer, Meg Ryan, Tom Skerritt, Kelly McGillis, Tom Cruise, and Anthony Edwards played in Top Gun.'

In [21]:
#cypher_chain.run("Who played in Top Gun Maverick?")